In [1]:
import sys
from importlib import reload
sys.path.insert(0, "..")
import utils.misc as misc
import distributed
import distributed.sharded_matrix
reload(distributed.sharded_matrix)
from distributed.sharded_matrix import ShardedMatrix, ShardedSymmetricMatrix
from distributed import kernel, matmul, bcd
import numpy as np
import scipy.linalg
import pywren
reload(pywren)
import math
import itertools
import time
import cloudpickle
import importlib
import sys
from sklearn.datasets import fetch_mldata
from itertools import groupby
from operator import itemgetter
from sklearn.kernel_approximation import RBFSampler, Nystroem
from sklearn import metrics
import sys
import boto3
import concurrent.futures as fs
reload(boto3)
from utils import linalg
from utils.misc import rreload
import scipy.linalg 




In [2]:
notebook_stdout = sys.stdout
notebook_stderr = sys.stderr
terminal_stdout = open('/dev/stdout', 'w')
terminal_stderr = open('/dev/stderr', 'w')

In [3]:
mnist = fetch_mldata('MNIST original', data_home="/tmp/mnist")
X_train = mnist.data[:60000].astype('float32')/255.0
y_train = mnist.target[:60000].astype('float32')
X_test = mnist.data[60000:].astype('float32')/255.0
y_test = mnist.target[60000:].astype('float32')


In [4]:
np.random.seed(0)
shuffled_idx = np.random.choice(X_train.shape[0], X_train.shape[0], replace=False)
X_train = X_train[shuffled_idx] 
y_train = y_train[shuffled_idx]
y_train_one_hot = np.eye(10)[y_train.astype('int')]


In [9]:
%time X_train_sharded_1 = ShardedMatrix("mnist_train_64",\
                                shard_size_0=4096,\
                                shard_size_1=64,\
                                data=X_train,\
                                bucket="vaishaalpywrenlinalg",\
                                reshard=True,\
                                n_jobs=32)

Sharding matrix..... of shape (60000, 784)
0 000000000004096409604096409640964096409640964096    4096               409640964096409681928192 4096409640964096  409640964096409640964096819281924096819281928192819281928192      81928192819281920    81924096               81928192819281921228812288     25619264320  512448576128768704012864019264256512320384      1228812288122881228864    640384               448576704768640    
320256128384  57651264019278476864192704256128320576384448      128192320256 



704448














51264076878412864   320384256192











81928192 12288  384 44812288
 448 512
81928192  122881228881928192122881228881921228812288122881228812288122881228812288     12288          12288512576122881228816384 16384122881638416384163841638416384163841638416384     16384          16384576640704768  12864019264256448384320576640 

  0512          704122881638416384  768784  192704256128320512448384640704  2048020480

64576









76816384   


640768   128647841638

In [10]:
%time X_test_sharded_1 = ShardedMatrix("mnist_test_64",\
                                shard_size_0=4096,\
                                shard_size_1=64,\
                                bucket="vaishaalpywrenlinalg",\
                                reshard=True,\
                                data=X_test,\
                                n_jobs=32)

Sharding matrix..... of shape (10000, 784)
000000000000409640964096409640964096040964096                   4096  40964096409640968192819281928192409640964096409640964096409640964096409640964096819281928192819281928192 40968192       819281928192                   8192  8192819281928192100001000010000   320641283202563841924480576512640704019225664128448 768384       100001000010000                   704  640576768512064128   3841281923843204482565126464057670476864256320128192512 784448       256192320

















768

70464078457664128192   384256320











8192 10000 384 448
8192 10000 448 512
8192 81928192  819281921000010000  10000 1000010000  704  512640 768576  768  576704
784640



CPU times: user 2.25 s, sys: 244 ms, total: 2.5 s
Wall time: 1.48 s


In [45]:
%time X_train_sharded_2 = ShardedMatrix("mnist_train_2",\
                                shard_size_0=4096,\
                                shard_size_1=784//2,\
                                bucket="vaishaalpywrenlinalg",\
                                data=X_train,\
                                reshard=True,\
                                n_jobs=32)
%time X_test_sharded_2 = ShardedMatrix("mnist_test_2",\
                                shard_size_0=4096,\
                                shard_size_1=784//2,\
                                bucket="vaishaalpywrenlinalg",\
                                data=X_test,\
                                reshard=True,\
                                n_jobs=32)

Sharding matrix..... of shape (60000, 784)
0 0 4096 409640961638481924096392     122888192 0819220480122888192 784      
39200039216384 
   784392392392



1228816384 20480 39212288245762867232768      39224576 368642048016384286723276836864  0  24576  40960 78428672  3920 0
 392368644915245056 7844096020480409605324828672450563276849152573445324857344392  0  392
   0
           784392 39260000 57344409605324849152 600004505624576450565734432768491523686453248784

392
 784    392         
392392
00
3923920392
392392000392              784784784392392392392784784392784392784784













CPU times: user 4.77 s, sys: 1.07 s, total: 5.84 s
Wall time: 2.55 s
Sharding matrix..... of shape (10000, 784)
4096 4096008192    819240960   3920392  
784392

4096 3928192  819210000784  
100000  392392 
784
CPU times: user 628 ms, sys: 80 ms, total: 708 ms
Wall time: 702 ms


In [53]:
%time X_train_sharded_3 = ShardedMatrix("mnist_test_2",\
                                shard_size_0=4096,\
                                shard_size_1=784//3,\
                                bucket="vaishaalpywrenlinalg",\
                                data=X_train,\
                                reshard=True,\
                                n_jobs=32)

Sharding matrix..... of shape (60000, 784)
0 4096 0 261
00  40964096  1228840960261783     4096819240968192409616384819240968192122888192122881228816384522784          20480    16384163842048016384204802048024576

8192052212288819201228881921228816384 12288163841638420480       24576245762867228672286722457628672           24576    20480204802457620480245762457628672     32768286727832617832615222617832610  5222615220       286722867232768327683276832768   

  
   783783    26178305222615220      261261784368645225227830784522261  783522783261       522783783  
  

 


784784



522784261783522783261   5225223276852278378478426132768783










  



 
368643276836864  36864783 3686436864368644096036864 
     7832614096040960409602610       7845220783522522261

   

783784261


40960 45056 0 261
40960 45056 261 522
40960 45056 522 783
40960 45056 783 784
45056 49152 0 45056 49152 261
261450564915245056    491525224915253248 
  522783  783784261491524915249152

    5324853248522532485

In [6]:
pwex = pywren.default_executor()
XXT_EXISTS = True

In [16]:
def check_if_sharded_matrix_is_close(local, sharded, num_samples):
    diff_norms = []
    max_norms = []
    block_idxs = sharded.block_idxs
    blocks = sharded.blocks
    idxs = np.random.choice(len(block_idxs), num_samples, replace=False)
    zipped_blocks = list(zip(block_idxs, blocks))
    for idx in idxs:
        bidx, block = zipped_blocks[idx]
        print(bidx, block)
        s3_block = sharded.get_block(*bidx)
        ((sidx_0,eidx_0),(sidx_1,eidx_1)) = block
        local_block = local[sidx_0:eidx_0, sidx_1:eidx_1]
        diff_norms.append(np.linalg.norm(local_block - s3_block))
        max_norms.append(np.max(np.abs(local_block - s3_block)))
    return np.mean(diff_norms), np.mean(max_norms)
    

In [117]:
reload(matmul)

%time XXT_sharded = matmul.compute_XYT_pywren(pwex, X=X_train_sharded_1, Y=X_train_sharded_1, tasks_per_job=2)

Submitting job for chunk 0 in axis 0 for chunk 0 in axis 1
Waiting job for chunk 0 in axis 0 for chunk 0 in axis 1
CPU times: user 7.18 s, sys: 684 ms, total: 7.86 s
Wall time: 41.8 s


In [115]:
%time XXT = X_train.dot(X_train.T)

CPU times: user 1min 51s, sys: 46.9 s, total: 2min 38s
Wall time: 15.1 s


In [118]:
XXT_sharded.get_block(0,0)

array([[ 131.90533447,   39.0460434 ,   37.10277939, ...,   79.78426361,
          36.28742981,   46.46697235],
       [  39.0460434 ,   63.25585938,   11.36120033, ...,   30.94794083,
          12.24796677,   14.17679501],
       [  37.10277939,   11.36120033,   59.01953125, ...,   36.60146332,
          28.01147079,   43.13497543],
       ..., 
       [  79.78426361,   30.94794083,   36.60146332, ...,  147.1544342 ,
          41.99662018,   50.75095749],
       [  36.28742981,   12.24796677,   28.01147079, ...,   41.99662018,
          65.81655884,   42.77620697],
       [  46.46697235,   14.17679501,   43.13497543, ...,   50.75095749,
          42.77620697,   87.78188324]], dtype=float32)

In [48]:
check_if_sharded_matrix_is_close(XXT, XXT_sharded, 10)

(0.0077534867, 3.1280517e-05)

In [49]:
X_test_sharded_1.get_block(1,0)

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]], dtype=float32)

In [9]:
reload(matmul)
%time XXT_test_sharded = matmul.compute_XYT_pywren(pwex, X=X_train_sharded_1, Y=X_test_sharded_1, tasks_per_job=2)

Submitting job for chunk 0 in axis 0 for chunk 0 in axis 1
Submitting job for chunk 1 in axis 0 for chunk 0 in axis 1
Waiting job for chunk 0 in axis 0 for chunk 0 in axis 1
Waiting job for chunk 1 in axis 0 for chunk 0 in axis 1
CPU times: user 3.78 s, sys: 416 ms, total: 4.2 s
Wall time: 36.7 s


In [10]:
XXT_test = X_test.dot(X_train.T)

In [57]:
sq_norms = np.power(np.linalg.norm(X_train, axis=1),2)

In [80]:
K_EXISTS = False

In [15]:
XXT_test_sharded.blocks

[((0, 4096), (0, 4096)),
 ((0, 4096), (4096, 8192)),
 ((0, 4096), (8192, 10000)),
 ((4096, 8192), (0, 4096)),
 ((4096, 8192), (4096, 8192)),
 ((4096, 8192), (8192, 10000)),
 ((8192, 12288), (0, 4096)),
 ((8192, 12288), (4096, 8192)),
 ((8192, 12288), (8192, 10000)),
 ((12288, 16384), (0, 4096)),
 ((12288, 16384), (4096, 8192)),
 ((12288, 16384), (8192, 10000)),
 ((16384, 20480), (0, 4096)),
 ((16384, 20480), (4096, 8192)),
 ((16384, 20480), (8192, 10000)),
 ((20480, 24576), (0, 4096)),
 ((20480, 24576), (4096, 8192)),
 ((20480, 24576), (8192, 10000)),
 ((24576, 28672), (0, 4096)),
 ((24576, 28672), (4096, 8192)),
 ((24576, 28672), (8192, 10000)),
 ((28672, 32768), (0, 4096)),
 ((28672, 32768), (4096, 8192)),
 ((28672, 32768), (8192, 10000)),
 ((32768, 36864), (0, 4096)),
 ((32768, 36864), (4096, 8192)),
 ((32768, 36864), (8192, 10000)),
 ((36864, 40960), (0, 4096)),
 ((36864, 40960), (4096, 8192)),
 ((36864, 40960), (8192, 10000)),
 ((40960, 45056), (0, 4096)),
 ((40960, 45056), (4096,

In [19]:
check_if_sharded_matrix_is_close(XXT_test.T, XXT_test_sharded, 10)

(5, 2) ((20480, 24576), (8192, 10000))
(8, 0) ((32768, 36864), (0, 4096))
(14, 0) ((57344, 60000), (0, 4096))
(3, 1) ((12288, 16384), (4096, 8192))
(7, 2) ((28672, 32768), (8192, 10000))
(13, 0) ((53248, 57344), (0, 4096))
(1, 0) ((4096, 8192), (0, 4096))
(12, 0) ((49152, 53248), (0, 4096))
(5, 1) ((20480, 24576), (4096, 8192))
(2, 1) ((8192, 12288), (4096, 8192))


(0.0001279945, 5.3405761e-06)

In [20]:
XXT_test.shape

(10000, 60000)

In [21]:
%time K_sharded = kernel.compute_rbf_kernel_pywren(pwex, XXT_sharded, X_train_sharded_1, X_train_sharded_2, gamma=1e-3, tasks_per_job=2)

NameError: name 'XXT_sharded' is not defined

In [22]:
%time K_test_sharded = kernel.compute_rbf_kernel_pywren(pwex, XXT_test_sharded, X_test_sharded_1, X_train_sharded_1, gamma=1e-3, tasks_per_job=2)

[0]
Submitting job for chunk 0
[0, 1]
0
(8192,)
Waiting for chunk 0
[0]
Submitting job for chunk 0
[0, 1]
0
(8192,)
Waiting for chunk 0
45
Submitting 22 jobs for chunk 0 out of 2 
Waiting for 22 jobs for chunk 0 out of 2 
Currently done 0 currently not done 22 
Currently done 0 currently not done 22 
Currently done 6 currently not done 16 
Submitting 1 jobs for chunk 1 out of 2 
Waiting for 1 jobs for chunk 1 out of 2 
Currently done 0 currently not done 1 
CPU times: user 5.42 s, sys: 820 ms, total: 6.24 s
Wall time: 39.1 s


In [150]:
K_test_sharded.get_block(0,0)

array([[ 0.95234227,  0.92599082,  0.95664865, ...,  0.88936526,
         0.88196307,  0.88325477],
       [ 0.87255418,  0.90196902,  0.89835656, ...,  0.919626  ,
         0.90432906,  0.87401301],
       [ 0.89937615,  0.88482445,  0.91426134, ...,  0.90527135,
         0.8892259 ,  0.88271278],
       ..., 
       [ 0.88662934,  0.86783308,  0.88363141, ...,  0.84814346,
         0.84800303,  0.88253587],
       [ 0.89118391,  0.88231534,  0.89253294, ...,  0.89453238,
         0.87122422,  0.88653368],
       [ 0.89457166,  0.8610006 ,  0.89910972, ...,  0.88164306,
         0.86802155,  0.8719545 ]], dtype=float32)

In [23]:
K_test = linalg.computeRBFGramMatrix(XTest=X_test, XTrain=X_train, gamma=1e-3)

In [28]:
check_if_sharded_matrix_is_close(K_test.T, K_test_sharded, num_samples=10)

(3, 1) ((12288, 16384), (4096, 8192))
(9, 0) ((36864, 40960), (0, 4096))
(8, 1) ((32768, 36864), (4096, 8192))
(12, 2) ((49152, 53248), (8192, 10000))
(4, 2) ((16384, 20480), (8192, 10000))
(14, 1) ((57344, 60000), (4096, 8192))
(7, 0) ((28672, 32768), (0, 4096))
(1, 2) ((4096, 8192), (8192, 10000))
(1, 1) ((4096, 8192), (4096, 8192))
(5, 2) ((20480, 24576), (8192, 10000))


(1.1065129e-06, 2.3841858e-08)

In [35]:
K_test[4096*2:4096*3,4096*8:4096*9].T

array([[ 0.9264366 ,  0.93271381,  0.89246243, ...,  0.91728157,
         0.91915786,  0.90167147],
       [ 0.93681061,  0.93997109,  0.88187027, ...,  0.90974921,
         0.89513421,  0.90411425],
       [ 0.8927415 ,  0.88944477,  0.93891454, ...,  0.90018755,
         0.91461205,  0.911538  ],
       ..., 
       [ 0.89084095,  0.90429646,  0.90148151, ...,  0.90511876,
         0.90009105,  0.89454436],
       [ 0.91821086,  0.93237615,  0.86226618, ...,  0.91872495,
         0.91562128,  0.92458278],
       [ 0.92331934,  0.92766863,  0.87903392, ...,  0.8809731 ,
         0.87708133,  0.88147402]], dtype=float32)

In [34]:
K_test_sharded.get_block(8,2)

array([[ 0.9264366 ,  0.93271381,  0.89246243, ...,  0.91728163,
         0.9191578 ,  0.90167147],
       [ 0.93681061,  0.93997109,  0.88187027, ...,  0.90974921,
         0.89513421,  0.90411419],
       [ 0.8927415 ,  0.88944477,  0.93891454, ...,  0.90018755,
         0.91461211,  0.911538  ],
       ..., 
       [ 0.89084095,  0.90429646,  0.90148151, ...,  0.90511876,
         0.90009105,  0.89454436],
       [ 0.91821086,  0.93237615,  0.86226618, ...,  0.91872495,
         0.91562128,  0.92458278],
       [ 0.92331934,  0.92766863,  0.87903392, ...,  0.8809731 ,
         0.87708133,  0.88147402]], dtype=float32)

In [18]:
y_enc = np.eye(10)[y_train.astype('int')]

In [15]:
XXT = XXT.astype('float64')

In [93]:
%time K = linalg.computeRBFGramMatrix(X_train, X_train, gamma=1e-3)

CPU times: user 2min 58s, sys: 38.3 s, total: 3min 36s
Wall time: 1min 18s


In [94]:
K

array([[ 0.99999994,  0.88952368,  0.88983488, ...,  0.87549806,
         0.88120699,  0.93134415],
       [ 0.88952368,  1.        ,  0.90524197, ...,  0.89721954,
         0.91604143,  0.91453439],
       [ 0.88983488,  0.90524197,  1.        , ...,  0.92135066,
         0.93387628,  0.91253197],
       ..., 
       [ 0.87549806,  0.89721954,  0.92135066, ...,  1.        ,
         0.91085505,  0.90198225],
       [ 0.88120699,  0.91604143,  0.93387628, ...,  0.91085505,
         1.        ,  0.91390967],
       [ 0.93134415,  0.91453439,  0.91253197, ...,  0.90198225,
         0.91390967,  1.        ]], dtype=float32)

In [69]:
import random
diff_norms = []
max_norms = []
for bidx, block in zip(block_idxs, blocks):
    if (random.random() > 0.75): continue
    s3_block = K_sharded.get_block(*bidx)
    ((sidx_0,eidx_0),(sidx_1,eidx_1)) = block
    local_block = K[sidx_0:eidx_0, sidx_1:eidx_1]
    diff_norms.append(np.linalg.norm(local_block - s3_block))
    max_norms.append(np.max(np.abs(local_block - s3_block)))
    print(np.mean(diff_norms))
    print(np.mean(max_norms))
    

7.85769022566e-05
1.86124413437e-07
7.86311769821e-05
1.88492473185e-07
7.86206647145e-05
1.95576728997e-07
7.86083408888e-05
1.9939509191e-07
7.85782299109e-05
1.91826753637e-07
7.85794947596e-05
1.94369887611e-07
7.85920960478e-05
1.91685352527e-07
7.85780958401e-05
1.92653506551e-07
7.68832079584e-05
1.91833839344e-07
7.70798157644e-05
1.94436902456e-07
7.72289110535e-05
1.92048793242e-07
7.73567145798e-05
1.9205611046e-07
7.74586618784e-05
1.91921168308e-07
7.7543596195e-05
1.93734761376e-07
7.76124637747e-05
1.92803643125e-07
7.76784148365e-05
1.92611770027e-07
7.77441049823e-05
1.91840978645e-07
7.77952484757e-05
1.93569649519e-07
7.78381029465e-05
1.94678863059e-07
7.78807006018e-05
1.95992614033e-07
7.71927700363e-05
1.96516781213e-07
7.72601633081e-05
1.96303918025e-07


KeyboardInterrupt: 

In [43]:
reload(bcd)
bcd.block_kernel_solve_debug(K, y_enc, block_size=4096, epochs=2, lambdav=1e-3, verbose=True)

Solving blocks 0 to 4095
solving block 0
Iteration 0, Training Accuracy 0.9529666666666666
Solving blocks 4096 to 8191
solving block 1
Iteration 1, Training Accuracy 0.96575
Solving blocks 8192 to 12287
solving block 2
Iteration 2, Training Accuracy 0.9706333333333333
Solving blocks 12288 to 16383
solving block 3
Iteration 3, Training Accuracy 0.9743
Solving blocks 16384 to 20479
solving block 4
Iteration 4, Training Accuracy 0.97755
Solving blocks 20480 to 24575
solving block 5
Iteration 5, Training Accuracy 0.9798166666666667
Solving blocks 24576 to 28671
solving block 6
Iteration 6, Training Accuracy 0.9816333333333334
Solving blocks 28672 to 32767
solving block 7
Iteration 7, Training Accuracy 0.9830833333333333
Solving blocks 32768 to 36863
solving block 8
Iteration 8, Training Accuracy 0.9848166666666667
Solving blocks 36864 to 40959
solving block 9
Iteration 9, Training Accuracy 0.9860166666666667
Solving blocks 40960 to 45055
solving block 10
Iteration 10, Training Accuracy 0.9

array([[  17.5854557 ,  -21.89850563,  -46.86743255, ...,   -5.76867144,
         -22.44168905,  -44.58497786],
       [  11.71114721,   -2.85437444,  -14.85082234, ...,   54.84312253,
           4.68510987,  -95.37683543],
       [  27.62378026,   35.59635285,   -3.45034974, ...,  -49.76394861,
         -80.29441818,  411.61310799],
       ..., 
       [  45.83917842,    8.78860365,  -17.16690181, ...,  141.05954682,
          21.94930975, -179.49950251],
       [  32.83178955,  -18.28629915,   22.2747839 , ...,  163.88959585,
         -64.71163312,  -58.06694923],
       [ -19.91840351,   32.64968444, -141.13107664, ...,   22.38889076,
          56.94977046,  -16.32474653]])

In [62]:
reload(bcd)
bcd.block_kernel_solve(K_sharded, y_enc, block_size=4096, epochs=1, lambdav=1e-3, num_blocks=len(K_sharded._blocks(0)))

Requesting: [1]
Special casing first epoch
Pre block accuracy is  0.097412109375
Residual Norm is  64.0
(4096, 4096)
(4096, 10)
solving block 0
Iteration 0, Training Accuracy 0.9529833333333333
Iter 0 took 6.044053316116333 seconds
Requesting: [2]
Special casing first epoch
Pre block accuracy is  0.944580078125
Residual Norm is  27.32012243
(4096, 4096)
(4096, 10)
solving block 1
Iteration 1, Training Accuracy 0.9657333333333333
Iter 1 took 2.424258232116699 seconds
Requesting: [3]
Special casing first epoch
Pre block accuracy is  0.959716796875
Residual Norm is  26.0606783764
(4096, 4096)
(4096, 10)
solving block 2
Iteration 2, Training Accuracy 0.97065
Iter 2 took 4.681600332260132 seconds
Requesting: [4]
Special casing first epoch
Pre block accuracy is  0.964599609375
Residual Norm is  25.4347967627
(4096, 4096)
(4096, 10)
solving block 3
Iteration 3, Training Accuracy 0.9743166666666667
Iter 3 took 3.7869856357574463 seconds
Requesting: [5]
Special casing first epoch
Pre block accu

(array([[ -11.39359469,  -26.5643392 ,  -44.83395258, ...,   22.25902939,
          -14.38540513,    6.50639841],
        [   1.81138846,    5.4756191 ,   -7.6311265 , ...,   24.10409267,
           -1.93207467,  -58.00393405],
        [  25.46729154,   20.4052448 ,    9.84006905, ...,  -57.56604857,
          -47.05517491,  268.98690829],
        ..., 
        [  21.83682204,    3.15764207,  -10.37781819, ...,   78.67849802,
           14.78557964,  -94.8809601 ],
        [  15.8425471 ,  -12.35863755,    7.18153325, ...,   93.15773153,
          -29.24825587,  -31.69386832],
        [  -6.86908814,   17.55341584,  -78.74682492, ...,   14.04814555,
           32.40458612,  -10.44194221]]),
 array([[ 0.94152002, -0.00170084, -0.03170156, ...,  0.06399064,
         -0.06068366,  0.15550835],
        [ 0.03986978, -0.03127713,  0.09388547, ..., -0.04474529,
         -0.05197447,  0.07890131],
        [ 0.04624429, -0.07894995,  0.05438627, ..., -0.09939903,
          0.03308949,  0.72626

In [19]:
%time model = scipy.linalg.solve(XXT, y_enc)

CPU times: user 1h 49min 22s, sys: 21min 15s, total: 2h 10min 38s
Wall time: 2min 39s


In [20]:
metrics.accuracy_score(np.argmax(XXT.dot(model), axis=1), y_train)

1.0

In [37]:
block_idxs = XXT_sharded.block_idxs
blocks = XXT_sharded.blocks

In [63]:
X_train_sharded_3.shard_size_1

261

In [68]:
import random
random.random()

0.4499722271499428

In [76]:
diff_norms = []
max_norms = []
for bidx, block in zip(block_idxs, blocks):
    s3_block = K_sharded.get_block(*bidx)
    ((sidx_0,eidx_0),(sidx_1,eidx_1)) = block
    local_block = K[sidx_0:eidx_0, sidx_1:eidx_1]
    diff_norms.append(np.linalg.norm(local_block - s3_block))
    max_norms.append(np.max(np.abs(local_block - s3_block)))
    print(np.mean(diff_norms))
    print(np.mean(max_norms))

7.85769022573e-05
1.86124413215e-07
7.86311769828e-05
1.88492473074e-07
7.86206647153e-05
1.95576728923e-07
7.86267352512e-05
1.84898541367e-07
7.86156620834e-05
1.90088869223e-07


KeyboardInterrupt: 

In [151]:
X_train_sharded._block_idxs(1)

[0, 1, 2, 3, 4, 5, 6, 7]

In [51]:
9.33738606592e-05/np.mean(s3_block)

2.6556017388208361e-06

In [150]:
np.all(np.isclose(XtX, future.download_matrix()))

True

In [65]:
future

In [64]:

reload(matmul)
matmul.compute_xtx_blockwise([(0,0)],future, X_train_sharded, other_block=1, axis=0)

(100, 4096)
True
XTX shape (100, 100)
0 0
0
4096
4096 4096


Exception: Incompatible block size: (100, 100) vs (4096, 4096)

In [161]:
X_train_sharded.get_block(1,0)
X_train_sharded.get_block(1,0)

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]], dtype=float32)

In [159]:
x_10 = block_kernel_solve(K_sharded, y_train_one_hot, max_iter=10, num_blocks=15, lambdav=1e-8)

NameError: name 'block_kernel_solve' is not defined

In [88]:
from scipy import linalg

In [89]:
linalg

<module 'scipy.linalg' from '/home/ubuntu/anaconda3/lib/python3.6/site-packages/scipy/linalg/__init__.py'>

In [86]:
scipy

AttributeError: module 'scipy' has no attribute 'linalg'